## Actividad 4 - Genetic Algorithm for the Knapsack Problem

### Genetic Algorithm

Team members:

- Luis Maximiliano López Ramírez Matricula: A08833321
- Annete Pamela Ruiz Abreu Matricula: A01423595
- Andrés Navarro Matricula: A00833287
- Andrea Axel Hernández Galgani Matricula: A00835225
- Jorge Raúl Rocha López Matricula: A01740816


### Context and how it works:

Genetic algorithms simulate the natural selection process, in which individuals (the solutions) "compete" to survive and improve over time through generations. The steps are:


- Initial Population: It starts with random solutions (individuals)
Selection: We have a function to evaluate how good a solution is (fitness). The best solutions are more likely to be selected and create a new generation.
Crossover: The selected solutions are combined to form new solutions. This process simulates reproduction, combining the genes of the parents to create sons.
- Mutation: With a low probability the solutions can experiment with modifications (mutations) to introduce variability.
- Replace: The new solutions replace the worst solutions of the population, and the process repeats itself.

1. Un armador tiene un carguero con capacidad de hasta 800 toneladas. El carguero
transporta contenedores de diferentes pesos para una determinada ruta. En la ruta
actual el carguero puede transportar algunos de los siguientes contenedores:


\[
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
\textbf{Contenedor} & c1 & c2 & c3 & c4 & c5 & c6 & c7 & c8 & c9 & c10 \\
\hline
\textbf{Peso [ton]} & 61 & 58 & 92 & 50 & 108 & 83 & 93 & 101 & 54 & 50 \\
\hline
\textbf{Beneficio [\$]} & 1100 & 1147 & 1442 & 1591 & 1078 & 1385 & 1777 & 1196 & 1753 & 1371 \\
\hline
\textbf{Contenedor} & c11 & c12 & c13 & c14 & c15 & c16 & c17 & c18 & c19 & c20 \\
\hline
\textbf{Peso [ton]} & 72 & 51 & 100 & 108 & 91 & 112 & 66 & 58 & 110 & 73 \\
\hline
\textbf{Beneficio [\$]} & 1517 & 1675 & 1193 & 1177 & 1365 & 1143 & 1314 & 1526 & 1470 & 1605 \\
\hline
\end{array}
\]


El analista de la empresa del armador desea determinar el env ́ıo (conjunto de contene-
dores) que maximiza el beneficio.

In [47]:
# Importing the required libraries
import time 
import random 

# Diccionario de contenedores con el formato {número: (peso, beneficio)}
contenedores = {
    1: (61, 1100),
    2: (58, 1147),
    3: (92, 1442),
    4: (50, 1591),
    5: (108, 1078),
    6: (83, 1385),
    7: (93, 1777),
    8: (101, 1196),
    9: (54, 1753),
    10: (50, 1371),
    11: (72, 1517),
    12: (51, 1675),
    13: (100, 1193),
    14: (108, 1177),
    15: (91, 1365),
    16: (112, 1143),
    17: (66, 1314),
    18: (58, 1526),
    19: (110, 1470),
    20: (73, 1605)
}

max_weight = 800

class GeneticAlgorithm():
    def __init__(self,data_dic, max_weight, population_size, n_crossover, mutation_rate, n_generations):
        self.data_dic = data_dic
        self.max_weight = max_weight
        self.population_size = population_size
        self.n_crossover = n_crossover
        self.mutation_rate = mutation_rate
        self.n_generations = n_generations
        self.population = []
        self.weight_population = []
        self.benefit_population = []
        self.fitness_population = []
        self.prob =[]
        self.penalty = 15000



    def CreateChromosome(self):
        chromosome = ''
        for i in range(len(self.data_dic)):
            chromosome += str(random.randint(0,1))
        return chromosome
    
    def InitializePopulation(self):
        counter =0
        while counter < self.population_size:
            chromosome = self.CreateChromosome()
            if self.IsValid(chromosome):
                self.population.append(chromosome)
                self.weight_population.append(self.CalculateWeight(chromosome))
                self.benefit_population.append(self.CalculateBenefit(chromosome))
                counter += 1
        
    
    def IsValid(self, chromosome):
        weight = 0
        for i in range(len(chromosome)):
            if chromosome[i] == '1':
                weight += self.data_dic[i+1][0]
        return weight <= self.max_weight
    
    def CalculateBenefit(self, chromosome):
        fitness = 0
        for i in range(len(chromosome)):
            if chromosome[i] == '1':
                fitness += self.data_dic[i+1][1]
        return fitness
    
    def CalculateBenefitPopulation(self):
        for u in range(len(self.population)):
            if self.IsValid(self.population[u]):
                self.benefit_population[u] = self.CalculateBenefit(self.population[u])
            else:
                self.benefit_population[u] = self.CalculateBenefit(self.population[u])-self.penalty
    
    def CalculateWeight(self, chromosome):
        weight = 0
        for i in range(len(chromosome)):
            if chromosome[i] == '1':
                weight += self.data_dic[i+1][0]
        return weight
    
    def CalculateWeightPopulation(self):
        for u in range(len(self.population)):
            self.weight_population[u] = self.CalculateWeight(self.population[u])
    
    
    def CalculateFitnessPopulation(self):
        self.fitness_population = []
        for i in range(len(self.population)):
            self.fitness_population.append(self.benefit_population[i]/sum(self.benefit_population))
    
    def CalculateProb(self):
        self.prob = []
        for i in range(len(self.population)):
            self.prob.append(self.fitness_population[i]/sum(self.fitness_population))
    
    def SelectParents(self):
        self.parents_index =[]
        self.prob_inferior_limit= []
        self.prob_superior_limit = []

        acum_li =0
        acum_ls =0

        for i in range(len(self.prob)):
            acum_ls += self.prob[i]
            self.prob_superior_limit.append(acum_ls)
            self.prob_inferior_limit.append(acum_li)
            acum_li += self.prob[i]
        while True:
            x = random.random()

            for i in range(len(self.prob)):
                if x >= self.prob_inferior_limit[i] and x <= self.prob_superior_limit[i] and i not in self.parents_index:
                    self.parents_index.append(i)
                    break
            if len(self.parents_index) == 2:
                break
    def CrossOver(self):
    
        self.sons = []
        self.weight_sons = []
        self.benefit_sons = []
        father = self.population[self.parents_index[0]]
        mother = self.population[self.parents_index[1]]

        son = father[:self.n_crossover] + mother[self.n_crossover:]
        daughter = mother[:self.n_crossover] + father[self.n_crossover:]


        # Mutation
        son = self.Mutation(son)
        daughter = self.Mutation(daughter)

        if(self.IsValid(son)):
            self.sons.append(son)
            self.weight_sons.append(self.CalculateWeight(son))
            self.benefit_sons.append(self.CalculateBenefit(son))

            # Adding a penalty
        if(self.IsValid(son) == False):
            self.sons.append(son)
            self.weight_sons.append(self.CalculateWeight(son))
            self.benefit_sons.append(self.CalculateBenefit(son)-self.penalty)
    

        if(self.IsValid(daughter)):
            self.sons.append(daughter)
            self.weight_sons.append(self.CalculateWeight(daughter))
            self.benefit_sons.append(self.CalculateBenefit(daughter))
            # Adding a penalty
        if(self.IsValid(daughter)==False):
            self.sons.append(daughter)
            self.weight_sons.append(self.CalculateWeight(daughter))
            self.benefit_sons.append(self.CalculateBenefit(daughter)-self.penalty)

            
    
    def ReplacePopulation(self):
        dic_benefit_pob = {index: benefit for index, benefit in enumerate(self.benefit_population)}
        two_least_values = sorted(dic_benefit_pob.items(), key=lambda item: item[1])[:2]
        worst_index, worst_benefit = two_least_values[0]
        worst_index2, worst_benefit2 = two_least_values[1]

        # Replacing the worst solutions with better ones
        if self.benefit_sons[0] > worst_benefit:
            self.population[worst_index] = self.sons[0]
            self.weight_population[worst_index] = self.weight_sons[0]
            self.benefit_population[worst_index] = self.benefit_sons[0]

        if self.benefit_sons[1] > worst_benefit2:
            self.population[worst_index2] = self.sons[1]
            self.weight_population[worst_index2] = self.weight_sons[1]
            self.benefit_population[worst_index2] = self.benefit_sons[1]

    def Mutation(self, chromosome):
        mutated_chromosome = ''

        for i in range(len(chromosome)):
            x = random.random()
            if x < self.mutation_rate:
                if chromosome[i] == '0':
                    mutated_chromosome += '1'
                else:
                    mutated_chromosome += '0'
            else:
                mutated_chromosome += chromosome[i]
        return mutated_chromosome
    
    def PrintBestPopulation(self):
        index_best_pob = self.benefit_population.index(max(self.benefit_population))

        print(f'Best solution: {self.population[index_best_pob]}')
        print(f'Best benefit: {self.benefit_population[index_best_pob]}')
        print('\n')

    def Run(self):
        self.InitializePopulation()

        for epoch in range(self.n_generations):
            self.CalculateBenefitPopulation()
            self.CalculateWeightPopulation()
            self.CalculateFitnessPopulation()
            self.CalculateProb()
            self.SelectParents()
            self.CrossOver()
            self.ReplacePopulation()
            if epoch % 100 == 0:
                print(f'Generation: {epoch}')
                self.PrintBestPopulation()



    
    

In [57]:
a = GeneticAlgorithm(contenedores, max_weight, 15, 10, 0.5, 1000)

In [58]:
a.Run()


Generation: 0
Best solution: 01010110111100010001
Best benefit: 14964


Generation: 100
Best solution: 01110101101100101001
Best benefit: 15990


Generation: 200
Best solution: 01010110110110001101
Best benefit: 16337


Generation: 300
Best solution: 01010110110110001101
Best benefit: 16337


Generation: 400
Best solution: 01010110110110001101
Best benefit: 16337


Generation: 500
Best solution: 01010110110110001101
Best benefit: 16337


Generation: 600
Best solution: 01010110110110001101
Best benefit: 16337


Generation: 700
Best solution: 01010110110110001101
Best benefit: 16337


Generation: 800
Best solution: 10110010110100101011
Best benefit: 16463


Generation: 900
Best solution: 10110010110100101011
Best benefit: 16463




In [59]:
print('Popultion: ')
a.population

Popultion: 


['00011100111100011101',
 '10111100110100101100',
 '01010110110110001101',
 '10110110011110000101',
 '01000100111101101101',
 '10110010100110101101',
 '10110010110100101011',
 '01010111111010001001',
 '01010110111000110101',
 '11011110101100100100',
 '11010110101100011100',
 '00010001111110011101',
 '01110000111110001101',
 '10011100111100100110',
 '01110101101100101001']

In [60]:
print('Weight of the population: ')
a.weight_population

Weight of the population: 


[777, 764, 736, 783, 764, 789, 791, 800, 794, 779, 758, 787, 724, 788, 791]

In [61]:
print('Benefit of the population: ')
a.benefit_population

Benefit of the population: 


[15958,
 15600,
 16337,
 16182,
 15835,
 16341,
 16463,
 15849,
 16180,
 15914,
 15928,
 15884,
 16134,
 15831,
 15990]

In [55]:
print('Fitness of the population: ')
a.fitness_population

Fitness of the population: 


[0.06608695652173913,
 0.06596186825380813,
 0.06647029153636638,
 0.06785836779985877,
 0.06640976495510945,
 0.06656309896096035,
 0.06612730757591043,
 0.06636134369010391,
 0.0662645011600928,
 0.06743871683647736,
 0.06643801069302936,
 0.06590537677796833,
 0.0671118732976899,
 0.06552607686875819,
 0.06947644507212751]